## How to use the UniRep mLSTM "babbler". This version demonstrates the 64-unit architecture which will be amenable to use on a personal laptop. For the 1900-unit production version see unirep_tutorial.ipynb

First, install the environment with your preferred method. We have tested the .yml conda load method on a conda python installation in Ubuntu 15.04, but not the others. We used pip to dump to requirements.txt which should be compatible with virtual env if you use python 3.6. 

To install something probably close enough to this environment you can use conda as follows:
```
conda create -n unirep python=3.6.0 tensorflow=1.3.0 jupyter pandas
source activate unirep
```

The most important version requirement is **tensorflow 1.3.0**. The others are likely fungible.

In [1]:
!aws s3 sync --no-sign-request --quiet s3://unirep-public/64_weights/ 64_weights/

In [2]:
import os
import tensorflow as tf
import numpy as np
from unirep import babbler64
tf.set_random_seed(42)
np.random.seed(42)

In [3]:
# Initialize the babbler. You need to provide the batch size you will use and the path to the weight directory.
batch_size = 12
b = babbler64(batch_size=batch_size, model_path="./64_weights")

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


The babbler needs to receive data in the correct format, a (batch_size, max_seq_len) matrix with integer values, where the integers correspond to an amino acid label at that position, and the end of the sequence is padded with 0s until the max sequence length to form a non-ragged rectangular matrix. We provide a formatting function to translate a string of amino acids into a list of integers with the correct codex:

In [4]:
seq = "MRKGEELFTGVVPILVELDGDVNGHKFSVRGEGEGDATNGKLTLKFICTTGKLPVPWPTLVTTLTYGVQCFARYPDHMKQHDFFKSAMPEGYVQERTISFKDDGTYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNFNSHNVYITADKQKNGIKANFKIRHNVEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSVLSKDPNEKRDHMVLLEFVTAAGITHGMDELYK"

In [5]:
np.array(b.format_seq(seq))

array([24,  1,  2,  4, 13,  6,  6, 21, 18,  8, 13, 16, 16, 14, 17, 21, 16,
        6, 21,  5, 13,  5, 16,  9, 13,  3,  4, 18,  7, 16,  2, 13,  6, 13,
        6, 13,  5, 15,  8,  9, 13,  4, 21,  8, 21,  4, 18, 17, 11,  8,  8,
       13,  4, 21, 14, 16, 14, 20, 14,  8, 21, 16,  8,  8, 21,  8, 19, 13,
       16, 10, 11, 18, 15,  2, 19, 14,  5,  3,  1,  4, 10,  3,  5, 18, 18,
        4,  7, 15,  1, 14,  6, 13, 19, 16, 10,  6,  2,  8, 17,  7, 18,  4,
        5,  5, 13,  8, 19,  4,  8,  2, 15,  6, 16,  4, 18,  6, 13,  5,  8,
       21, 16,  9,  2, 17,  6, 21,  4, 13, 17,  5, 18,  4,  6,  5, 13,  9,
       17, 21, 13,  3,  4, 21,  6, 19,  9, 18,  9,  7,  3,  9, 16, 19, 17,
        8, 15,  5,  4, 10,  4,  9, 13, 17,  4, 15,  9, 18,  4, 17,  2,  3,
        9, 16,  6,  5, 13,  7, 16, 10, 21, 15,  5,  3, 19, 10, 10,  9,  8,
       14, 17, 13,  5, 13, 14, 16, 21, 21, 14,  5,  9,  3, 19, 21,  7,  8,
       10,  7, 16, 21,  7,  4,  5, 14,  9,  6,  4,  2,  5,  3,  1, 16, 21,
       21,  6, 18, 16,  8

I also provide a function that will check your Amino Acid sequences don't contain any characters which will break the babbler.

In [6]:
b.is_valid_seq(seq)

True

If you are into it, you could do your own data flow as long as you ensure that the data format is obeyed. Alternatively, you could use the data flow we implemented for babbler training, which happens in the tensorflow graph. It reads from a file of integer sequences, shuffles them around, collects them into groups of similar length (to minimize padding waste) and pads them to the max_length. We'll show you how to do that below:

In [7]:
# Before you can train your model, sequences need to be saved in the correct format
# suppose we have a new-line seperated file of AA sequences, seqs.txt, and we want to format them.
with open("seqs.txt", "r") as source:
    with open("formatted.txt", "w") as destination:
        for i,seq in enumerate(source):
            seq = seq.strip()
            if b.is_valid_seq(seq):
                formatted = ",".join(map(str,b.format_seq(seq)))
                destination.write(formatted)
                destination.write('\n')
            else:
                raise ValueError("Sequence {0} is not a valid sequence.".format(i))


In [8]:
# This is what the integer format looks like
!head -n1 formatted.txt

24,1,2,4,13,6,6,21,18,8,13,16,16,14,17,21,16,6,21,5,13,5,16,9,13,3,4,18,7,16,2,13,6,13,6,13,5,15,8,9,13,4,21,8,21,4,18,17,11,8,8,13,4,21,14,16,14,20,14,8,21,16,8,8,21,8,19,13,16,10,11,18,15,2,19,14,5,3,1,4,10,3,5,18,18,4,7,15,1,14,6,13,19,16,10,6,2,8,17,7,18,4,5,5,13,8,19,4,8,2,15,6,16,4,18,6,13,5,8,21,16,9,2,17,6,21,4,13,17,5,18,4,6,5,13,9,17,21,13,3,4,21,6,19,9,18,9,7,3,9,16,19,17,8,15,5,4,10,4,9,13,17,4,15,9,18,4,17,2,3,9,16,6,5,13,7,16,10,21,15,5,3,19,10,10,9,8,14,17,13,5,13,14,16,21,21,14,5,9,3,19,21,7,8,10,7,16,21,7,4,5,14,9,6,4,2,5,3,1,16,21,21,6,18,16,8,15,15,13,17,8,3,13,1,5,6,21,19,4


Notice that by default format_seq does not include the stop symbol (25) at the end of the sequence. This is the correct behavior if you are trying to train a top model, but not if you are training a babbler.

Now we can use my custom function to bucket, batch and pad sequences  from formatted.txt (which has the correct integer codex after calling)
babbler.format_seq(). The bucketing occurs in the graph. 

What is bucketing? Specify a lower and upper bound, and interval. All sequences less than lower or greater than upper will be batched together. Interval defines the "sides" of buckets between these bounds. Don't pick a small interval for a small dataset because
the function will just repeat a sequence if there are not enough to
fill a batch. All batches are the size you passed when initializing the babbler.
What else is this doing? 
- Shuffling the sequences by randomly sampling from a 10000 sequence buffer
- Automatically padding the sequences with zeros so the returned batch is a perfect rectangle
- Automatically repeating the dataset (you will need synthetic epochs)

In [9]:
bucket_op = b.bucket_batch_pad("formatted.txt", interval=1000) # Large interval

Inconveniently, this does not make it easy for a value to be associated with each sequence and not lost during shuffling. You can get around this by just prepending every integer sequence with the sequence label (eg, every sequence would be saved to the file as "{brightness value}, 24, 1, 5,..." and then you could just index out the first column after calling the bucket_op. Please reach out if you have questions on how to do this.

In [10]:
# Now that we have the bucket_op, we can simply sess.run() it to get
# a correctly formatted batch

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    batch = sess.run(bucket_op)
    
print(batch)
print(batch.shape)

[[24  1  7 ...  0  0  0]
 [24  1 15 ...  0  0  0]
 [24  1 15 ...  0  0  0]
 ...
 [24 16  4 ...  0  0  0]
 [24  1 15 ...  0  0  0]
 [24  1 16 ... 15 15  3]]
(12, 905)


You can look back and see that the batch_size we passed to __init__ is indeed 12, and the second dimension must be the longest sequence included in this batch. Now we have the data flow setup (note that as long as your batch looks like this, you don't need my flow), so we can proceed to implementing the graph. The module returns all the operations needed to feed in sequence and get out trainable representations.

In [11]:
final_hidden, x_placeholder, batch_size_placeholder, seq_length_placeholder, initial_state_placeholder = (
    b.get_rep_ops())

In [12]:
# final_hidden should be a batch_size x rep_dim matrix
# Lets say I want to train a basic feed-forward network as the top
# model, doing regression with MSE loss, Adam optimizer
y_placeholder = tf.placeholder(tf.float32, shape=[None,1], name="y")
initializer = tf.contrib.layers.xavier_initializer(uniform=False)

with tf.variable_scope("top"):
    prediction = tf.contrib.layers.fully_connected(
        final_hidden, 1, activation_fn=None, 
        weights_initializer=initializer,
        biases_initializer=tf.zeros_initializer()
    )

loss = tf.losses.mean_squared_error(y_placeholder, prediction)
    
# You can specifically train the top model first
learning_rate=.1
top_variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="top")
optimizer = tf.train.AdamOptimizer(learning_rate)
top_only_step_op = optimizer.minimize(loss, var_list=top_variables)
all_step_op = optimizer.minimize(loss)

In [13]:
# Notice that one of the placeholder is seq_length_placeholder.
# We need to compute the lengths of the sequences in each batch so
# that we can index out the the correct final hidden.
def nonpad_len(batch):
    nonzero = batch > 0
    lengths = np.sum(nonzero, axis=1)
    return lengths

nonpad_len(batch)

array([239, 342, 832, 445, 376, 376, 239, 232, 833, 361, 230, 905])

In [14]:
# toy example where we learn to predict 42 just training the top
y = [[42]]*batch_size
num_iters = 10
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(num_iters):
        batch = sess.run(bucket_op)
        length = nonpad_len(batch)
        loss_, __, = sess.run([loss, top_only_step_op],
                             feed_dict={
                                 x_placeholder: batch,
                                 y_placeholder: y,
                                 batch_size_placeholder: batch_size,
                                 seq_length_placeholder:length,
                                 initial_state_placeholder:b._zero_state
                             }
                             )
        print("Iteration {0}: {1}".format(i, loss_))

Iteration 0: 1734.759765625
Iteration 1: 1623.6226806640625
Iteration 2: 1511.0751953125
Iteration 3: 1416.2100830078125
Iteration 4: 1317.2991943359375
Iteration 5: 1211.9937744140625
Iteration 6: 1132.4569091796875
Iteration 7: 1045.625244140625
Iteration 8: 853.5608520507812
Iteration 9: 871.4636840820312


In [15]:
# And another where we train the recurrent part as well
# This (64 dim) works on my laptop. I can save the trained weights 
# as long as I do it within the same session.
output_dir = "64_predict_42_weights"
os.makedirs(output_dir, exist_ok=True)

y = [[42]]*batch_size
num_iters = 5
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(num_iters):
        batch = sess.run(bucket_op)
        length = nonpad_len(batch)
        loss_, __, = sess.run([loss, all_step_op],
                             feed_dict={
                                 x_placeholder: batch,
                                 y_placeholder: y,
                                 batch_size_placeholder: batch_size,
                                 seq_length_placeholder:length,
                                 initial_state_placeholder:b._zero_state
                             }
                             )
        print("Iteration {0}: {1}".format(i, loss_))
    b.dump_weights(sess, dir_name=output_dir)

Iteration 0: 1734.759765625
Iteration 1: 1323.5928955078125
Iteration 2: 1172.90087890625
Iteration 3: 990.4606323242188
Iteration 4: 808.5928344726562
embed_matrix:0
[[ 0.5660815   0.8955147  -0.1885159   0.6356604  -0.6028085  -0.7326951
   0.20087385  0.21737194  0.47095132 -0.91131735]
 [-1.3334079   0.2264685  -0.7375298  -1.3277719  -0.40845445 -0.00811731
   1.1949555  -0.44037223 -0.47893474  1.8256454 ]
 [ 0.15790248 -0.5947473   0.9383942   0.30826464  1.078416    1.1427243
  -0.50845486 -0.42546564 -1.6841255   1.5630785 ]
 [ 1.7446326   0.564302    1.1153563   1.4399172   0.17611572 -0.8778575
  -0.8982393   0.20469265 -0.9477057   0.20154099]
 [ 0.662779    0.78323305  0.9347842  -0.40930912 -0.6207866   0.3517494
  -0.7100488  -0.42688814  0.5588627   1.6817728 ]
 [ 1.3529816  -0.11552777 -0.63151    -0.5929775  -0.45382506  0.96362585
  -0.93400884 -1.0272582  -0.1974521  -0.17272826]
 [ 1.3828369   1.4690058   0.81367016  1.0096147  -0.3611145   0.5928308
   0.0668923  

rnn/mlstm_stack/mlstm_stack2/mlstm_stack2/gx:0
[1.1400656  0.5916189  1.1283973  1.2596868  2.0975266  0.85378236
 2.3082678  1.365086   1.3827409  1.6842247  1.2878246  0.7994257
 1.4022663  1.462476   1.8074237  1.202274   0.85268354 1.6855634
 1.9070449  0.8012539  1.4721088  1.0205564  1.443837   1.5199773
 1.0357943  0.6514754  0.81160647 1.4745808  1.4263434  2.0842772
 0.8482999  0.8260567  1.273469   1.4864529  0.6691973  0.8821564
 1.4619867  1.4138715  1.6615324  0.7189499  0.7842356  1.6019431
 0.8537523  0.90540063 1.444757   1.3996823  1.3074592  1.9944938
 1.360171   0.8173645  1.1406701  1.1629215  1.6286137  1.3939836
 0.3310691  1.5561624  2.4146235  1.8026377  2.312918   1.304507
 0.85909414 0.73227984 1.5913451  1.3063743  0.6238285  1.183035
 0.6195258  1.0076278  0.8181681  0.78968084 0.63274145 1.2967746
 1.4717447  1.4093164  0.755982   1.3968204  1.4391793  0.6763977
 0.4590506  1.4374676  0.63453275 1.1118982  1.2580041  0.91691625
 1.2110817  1.2005402  0.9721